In [1]:
from baselines.common import set_global_seeds, tf_util as U
from baselines import bench
import gym, logging
import roboschool
from baselines import logger
from baselines.ppo1 import mlp_policy, pposgd_simple
from base_line_model.mlp import MlpPolicy_new
from base_line_model.TRPO_agent import TRPO_agent_new

In [2]:
seed = 1
logger.configure()
U.make_session(num_cpu=16).__enter__()
set_global_seeds(seed)

Logging to /tmp/openai-2018-04-20-03-44-58-216472


# set up agents

In [3]:
env2 = gym.make('RoboschoolHopper-v1')
env2 = bench.Monitor(env2, logger.get_dir(),allow_early_resets=True)
env2.seed(seed)
gym.logger.setLevel(logging.WARN)
class pargm(object):
    def __init__(self):
        self.timesteps_per_batch = 1024 # what to train on
        self.max_kl = 0.01
        self.cg_iters = 10
        self.gamma = 0.995
        self.lam =  0.97# advantage estimation
        self.entcoeff=0.0
        self.cg_damping=0.1
        self.vf_stepsize=1e-3
        self.vf_iters =5
        self.max_timesteps = 1e4
        self.max_episodes=0
        self.max_iters=0  # time constraint
        self.callback=None


def policy_fn(name, ob_space, ac_space):
        return MlpPolicy_new(name=name, ob_space=ob_space, ac_space=ac_space,
            hid_size=128, num_hid_layers=2)
    
parg = pargm()
agent1 = TRPO_agent_new('pi1', env2, policy_fn, parg)

[2018-04-20 03:44:58,242] Making new env: RoboschoolHopper-v1


Init param sum 8.43296


# Training

In [4]:
agent1.learn()

********** Iteration 0 ************
sampling
done in 1.643 seconds
computegrad
done in 0.029 seconds
cg
      iter residual norm  soln norm
         0      0.118          0
         1     0.0393     0.0754
         2     0.0222      0.159
         3       0.01      0.203
         4    0.00531      0.231
         5    0.00402      0.349
         6    0.00141      0.417
         7   0.000919      0.453
         8    0.00018      0.476
         9   0.000304      0.478
        10   5.57e-05      0.485
done in 0.099 seconds
Expected: 0.038 Actual: 0.036
Stepsize OK!
vf
done in 0.256 seconds
-------------------------------
| EpLenMean       | 12.6      |
| EpRewMean       | 13.9      |
| EpThisIter      | 86        |
| EpisodesSoFar   | 86        |
| TimeElapsed     | 2.07      |
| TimestepsSoFar  | 1021      |
| entloss         | 0.0       |
| entropy         | 4.25682   |
| ev_tdlam_before | -0.0268   |
| meankl          | 0.0072308 |
| optimgain       | 0.0360956 |
| surrgain        | 0.0

Stepsize OK!
vf
done in 0.260 seconds
-------------------------------
| EpLenMean       | 10.4      |
| EpRewMean       | 16.7      |
| EpThisIter      | 103       |
| EpisodesSoFar   | 828       |
| TimeElapsed     | 15.3      |
| TimestepsSoFar  | 8188      |
| entloss         | 0.0       |
| entropy         | 4.25682   |
| ev_tdlam_before | 0.746     |
| meankl          | 0.0065794 |
| optimgain       | 0.0344023 |
| surrgain        | 0.0344023 |
-------------------------------
********** Iteration 8 ************
sampling
done in 1.544 seconds
computegrad
done in 0.007 seconds
cg
      iter residual norm  soln norm
         0     0.0921          0
         1     0.0271     0.0454
         2     0.0104     0.0818
         3     0.0208      0.137
         4    0.00918      0.195
         5    0.00755      0.227
         6    0.00715      0.276
         7     0.0049        0.3
         8    0.00355      0.344
         9    0.00408      0.392
        10     0.0014      0.416
done in 0.0

# Evaluation

In [5]:
env = gym.make("RoboschoolHopper-v1")
for i in range(10):
    score = 0
    obs = env.reset()
    done = False
    itr = 0
    do = False
    while done == False:   
        a = agent1.action_ev(obs)
        obs, r, done, _ = env.step(a)
        if done:
            do = True

        score += r
    print(score)

33.86384692986467
36.93324909845396
34.26638970317727
41.03746049799665
37.8342052278691
32.92402653788595
43.39411796944478
40.41015319144935
37.784243869934286
41.27363742785528


In [6]:
agent1.save_data('hopper_mem2')

In [7]:
#agent1.restore('hopper_mem2')